# Contexto e Dados

### Contexto
Os datasets em conjunto registram dados de pescas de mais de 200 espécies de peixes e conchas no Ocêano Atlântico. Esses dados foram oficialmente registrados por 20 países que participam do Conselho Internacional de Exploração Marítima (ICES) entre 2006 e 2014.<br>
Fonte: https://www.kaggle.com/datasets/victorgenin/ices-fish-catch?select=species.txt

### Dados
Exitsem 4 fontes de dados, são elas:
- **country_codes.csv**
  <br>Tabela contendo metadados dos países. 6 atributos: `code`, `description`, `codeType`, `deprecated`, `created`, `modified`.
- **fish_catches.csv**
  <br>Tabela contendo registros de captura. 24 atributos: `species`, `area`, `units`, `country`, `2014`, `2013`, `2012`, `2011`, `2010`, `2009`, `2008`, `2007`, `2006`, e o restante inacessível
- **fishing_area.xml**
  <br>Objeto contendo dados das subdivisões das áreas do oceano Atlântico (schema ainda não identificado)
- **species.txt**
  <br>Arquivo texto contendo informação das espécies de peixes. 11 atributos: `ISSCAAP`, `TAXOCODE`, `3A_CODE`, `Scientific_name`, `English_name`, `French_name`, `Spanish_name`, `Author`, `Family`, `Order`, `Stats_data`

# Planejamento

1. As 4 base de dados deverão ser ingeridas e armazenadas
2. Para cade base de dado originalmente do tipo csv ou txt, serão criados objetos do tipo Dataframe com o parser padrão do leitor desses arquivos
3. Será escolhido uma biblioteca ou criado uma função simples para parsear a fonte de dados de formato xml, e transformada em Dataframe
4. Para cada base de dados, já no formato Dataframe, será aplicado funções de tratamento de dados (correção de texto, tratamento de nulos, formatação, etc)
5. Será avaliado a necessidade de remodelagem das bases para facilitar o uso analítico. Se viável, uma remodegalem será feita em cima das 4 tabelas
6. Criação de um dicionário de dados para cada fonte, em formato yaml

# EDA

In [ ]:
df = pd.read_csv('/content/final.csv').drop(columns=['Unnamed: 0'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352196 entries, 0 to 352195
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   local            352196 non-null  object 
 1   loc_type         352196 non-null  object 
 2   country          352196 non-null  object 
 3   year             352196 non-null  int64  
 4   aka              352196 non-null  object 
 5   quantity         352196 non-null  float64
 6   units            352196 non-null  object 
 7   scientific_name  352196 non-null  object 
 8   family           352196 non-null  object 
 9   order            352196 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 26.9+ MB


A maioria das variáveis são do tipo `object`, que representa texto.
As outras duas restantes variáveis representam escalares.
`year` é do tipo `int` e `quantity` é do tipo `float`, que considera decimais

In [ ]:
df[df.duplicated()]

,local,loc_type,country,year,aka,quantity,units,scientific_name,family,order


In [ ]:
df.nunique()

local                94
loc_type              5
country              23
year                  9
aka                 887
quantity           8640
units                 1
scientific_name     902
family              216
order                70
dtype: int64

Todas 352.701 linhas são únicas, porém podemos observar que existem bastantes valores repetidos em algumas colunas:


*   Ano, com 9 valores
*   Localizações, com 94 valores
*   Order, com 70 valores
*   Países, com 23 valores
*   Unidade, com apenas um valor

Com isso concluímos que esses dados englobam pesca de 902 espécies de peixes, em 23 países diferentes, durante 9 anos (9 valores distintos e sabemos que os dados foram coletados em um espaço contínuo de tempo).
Dessas 902 espécies, identificamos que existem 887 nomes em inglês para aquelas espécies, portanto devem haver N registros de nomes diferentes porém se referindo a peixes da mesma espécie.



In [ ]:
df.isnull().sum()

local              0
loc_type           0
country            0
year               0
aka                0
quantity           0
units              0
scientific_name    0
family             0
order              0
dtype: int64

### Visualização das variáveis

#### Quantidade (TLW: Tonnes of Live Weight = Toneladas de peixe fora d'água)
- Histograma de frequências
- Quantidade total por país e ano
- Quantidade total por espécie de peixe e ano

In [ ]:
df['quantity'].max(), df['quantity'].mean(), df['quantity'].median()

(1070437.0, 708.7845319083692, 0.0)

In [ ]:
import plotly.express as px
fig = px.histogram(df, x="quantity", nbins=50)
fig.show()

Existem valores muito discrepantes. Iremos plotar um histograma considerando 95% dos dados.

In [ ]:
df_plot1 = df[df['quantity']<df['quantity'].quantile(0.95)]

In [ ]:
fig = px.histogram(df_plot1, x="quantity")
fig.show()

Existem quase 250.000 linhas com quantidade igual à zero

Iremos retirar os valores zero e novamente os 5% maiores valores de quantidade

In [ ]:
df_plot1 = df_plot1[(df_plot1['quantity'] > 0) & (df_plot1['quantity']<df_plot1['quantity'].quantile(0.95))]
px.histogram(df_plot1, x="quantity").show()

In [ ]:
px.histogram(df_plot1, x="quantity", color="year").show()

Aparentemente a frequência de cada valor observado (quantidade de TLW/Total live weight/peso pescados) é bem distribuida por ano (cor) e também parece seguir uma distribuição Exponencial

In [ ]:
## Gráfico TWL vs País (quebrado por ano)

In [ ]:
df_groupby_country = df[['country', 'quantity', 'year']].groupby(by=['country', 'year']).agg(lambda x: sum(x)).reset_index()
df_groupby_country2 = df_groupby_country.assign(qty_by_ct=df_groupby_country.groupby('country')['quantity'].transform('sum')).sort_values('qty_by_ct', ascending = False)

In [ ]:
px.bar(df_groupby_country2, x='country', y='quantity',  barmode='group', color='year').show()

A Noruega é o país que mais pesca, em termos de peso. Além disso, parece que em 2013 esse país reverteu a tendência dos anos anteriores, que diminuia o peso por ano. Em seguida, a Islândia e a Russia são países com valores expressivos, acima de 20 milhões de toneladas entre 2006 e 2014.

In [ ]:
## Gráfico TWL vs Espécie (quebrado por ano)

In [ ]:
df_groupby_name = df[['scientific_name', 'quantity', 'year']].groupby(by=['scientific_name', 'year']).agg(lambda x: sum(x)).reset_index()
df_groupby_name2 = df_groupby_name.assign(qty_by_sp=df_groupby_name.groupby('scientific_name')['quantity'].transform('sum')).sort_values('qty_by_sp', ascending = False)

In [ ]:
px.bar(df_groupby_name2[df_groupby_name2['qty_by_sp']>800000], x='scientific_name', y='quantity',  barmode='group', color='year').show()

Os peixes mais pescados são os: `Clupea harengus` e `Gadus morhua`

### Investigando se a granularidade do dataset está correta
Hipótese: Haver diferentes granularidades decorrente da área/local, que incorerrá em valores agregados duplicadamente. Como desconhecemos o sistema que registrou esses dados, existe a possibilidade de haver uma entrada agregada para cada local no nível área. Essa suspeita existe pois os valores de `quantity` parecem muito altos quando `loc_type = area`. Ou seja, é possível que registros em locais dentro daquela area (`loc_type` = `subarea`/`division`/`subdivision`/`unit`) estejam sendo agregados e criado um novo registro no nível mais alto.

In [ ]:
## Gráfico

In [ ]:
df_with_loctype = df[['quantity', 'year', 'loc_type']].groupby('loc_type')['quantity'].transform('sum')
df_with_loctype.head(20)

0     77453761.0
1     77453761.0
2     77453761.0
3     77453761.0
4     77453761.0
5     77453761.0
6     77453761.0
7     77453761.0
8     77453761.0
9     77453761.0
10    77453761.0
11    77453761.0
12    77453761.0
13    77453761.0
14    77453761.0
15    77453761.0
16    77453761.0
17    77453761.0
18    77453761.0
19    77453761.0
Name: quantity, dtype: float64